# [fileinput]通过重定向/管道/文件接受输入

在命令行中传递一个文件名给该脚本

In [60]:
%%file filein.py

import fileinput
import subprocess

# ===== sys ======
# python "...\filein.py" FileinputTest.txt
import sys
a=sys.argv[0]
b=sys.argv[1]

print("filename:",a)
print("param1:",b)

# ===== input =====
file = input("請輸入...")
print(file)

with fileinput.input(file) as f_input:
    for line in f_input:
        print(line, end='')
        
# 暫停        
subprocess.call("pause",shell=True)

Overwriting filein.py


In [61]:
# ============ 開新 CONSOLE ============
# ------------ Run the server ------------

import os
import subprocess
# os.path.abspath {本黨位置}: D:\Google 雲端硬碟\learn\線程調用\TestOS.py
# os.path.dirname {目錄} : D:\Google 雲端硬碟\learn\線程調用
BASE_DIR = os.path.dirname(os.path.abspath('__file__'))

# 透過 cmd 呼叫
DIR = os.path.join(BASE_DIR, 'filein.py')
cmd = "python " + f'"{DIR}"' # + ' FileinputTest.txt' # sys.argv[1]

# 印出 cmd DIR
print(cmd,BASE_DIR,sep='\n')

#  CONSOLE混雜
#os.system(cmd)
#subprocess.call(cmd)

#  NEW 一個 CONSOLE
subprocess.Popen(cmd, creationflags=subprocess.CREATE_NEW_CONSOLE)

# 暫停(需在要執行的 py 上)
#subprocess.call("pause",shell=True)
#os.system("pause")

python "D:\CODE\GitHub\py\資料結構\py3-cookbook\脚本编程与系统管理\filein.py" FileinputTest.txt
D:\CODE\GitHub\py\資料結構\py3-cookbook\脚本编程与系统管理


`fileinput.input()` 创建并返回一个 `FileInput` 类的实例。 

该实例除了拥有一些有用的帮助方法外，它还可被当做一个上下文管理器使用。 

通过将它作为一个上下文管理器使用，可以确保它不再使用时文件能自动关闭。

因此，整合起来，如果我们要写一个打印多个文件输出的脚本，那么我们需要在输出中包含文件名和行号

In [34]:
import fileinput

with fileinput.input('FileinputTest.txt') as f:
    for line in f:
        print(f.filename(), f.lineno(), line, end='') 

FileinputTest.txt 1 1: AAAAA
FileinputTest.txt 2 2: BBBBB
FileinputTest.txt 3 3: CCCCC
FileinputTest.txt 4 4: DDDDD


# [raise SystemExit('It failed!')]终止程序并给出错误信息

抛出一个 `SystemExit` 异常，使用错误消息作为参数。

例如：

```py
raise SystemExit('It failed!')
```

它会将消息在 `sys.stderr` 中打印，然后程序以状态码1退出

当你想要终止某个程序时，你可能会像下面这样写：

```py
import sys
sys.stderr.write('It failed!\n')
raise SystemExit(1)
```

如果你直接将消息作为参数传给 `SystemExit()` ，那么你可以省略其他步骤， 

比如 `import` 语句或将错误消息写入 `sys.stderr`

# [argparse]解析命令行选项

首先要创建一个 `ArgumentParser` 实例， 并使用 `add_argument()` 方法声明你想要支持的选项。 

在每个 `add_argument()` 调用中， `dest` 参数指定解析结果被指派给属性的名字。

`metavar` 参数被用来生成帮助信息。

`action` 参数指定跟属性对应的处理逻辑， 通常的值为 `store` ,被用来存储某个值或将多个参数值收集到一个列表中。 

---

`nargs='*'` 下面的参数收集所有剩余的命令行参数到一个列表中。

在本例中它被用来构造一个文件名列表：

```py
parser.add_argument(dest='filenames',metavar='filename', nargs='*')
```

---

`action='store_true'` 下面的参数根据参数是否存在来设置一个 `Boolean` 标志：

```py
parser.add_argument('-v', dest='verbose', action='store_true',
                    help='verbose mode')
```

---

`action='store'` 下面的参数接受一个单独值并将其存储为一个字符串：

```py
parser.add_argument('-o', dest='outfile', action='store',
                    help='output file')
```

---

`action='append'` 下面的参数说明允许某个参数重复出现多次，并将它们追加到一个列表中去。 

`required` 标志表示该参数至少要有一个。

`-p` 和 `--pat` 表示两个参数名形式都可使用。

```py
parser.add_argument('-p', '--pat',metavar='pattern', required=True,
                    dest='patterns', action='append',
                    help='text pattern to search for')
```

---

最后， `choices={'slow','fast'}` 下面的参数说明接受一个值，但是会将其和可能的选择值做比较，以检测其合法性：

```py
parser.add_argument('--speed', dest='speed', action='store',
                    choices={'slow','fast'}, default='slow',
                    help='search speed')
```

---

一旦参数选项被指定，你就可以执行 `parser.parse()` 方法了。 它会处理 `sys.argv` 的值并返回一个结果实例。 

每个参数值会被设置成该实例中` add_argument()` 方法的 `dest` 参数指定的属性值。


In [14]:
%%file search.py
'''
Hypothetical command-line tool for searching a collection of
files for one or more text patterns.
'''

import os

import argparse
parser = argparse.ArgumentParser(description='Search some files')

parser.add_argument(dest='filenames',metavar='filename', nargs='*')

# required=True 必填
parser.add_argument('-p', '--pat',metavar='pattern', required=True,
                    dest='patterns', action='append',
                    help='text pattern to search for')

parser.add_argument('-v', dest='verbose', action='store_true',
                    help='verbose mode')

parser.add_argument('-o', dest='outfile', action='store',
                    help='output file')

parser.add_argument('--speed', dest='speed', action='store',
                    choices={'slow','fast'}, default='slow',
                    help='search speed')

args = parser.parse_args()

# Output the collected arguments
print(args.filenames)
print(args.patterns)
print(args.verbose)
print(args.outfile)
print(args.speed)

os.system("pause")

Overwriting search.py


In [24]:
# ============ 開新 CONSOLE ============
# ------------ Run the server ------------

import os
import subprocess
# os.path.abspath {本黨位置}: D:\Google 雲端硬碟\learn\線程調用\TestOS.py
# os.path.dirname {目錄} : D:\Google 雲端硬碟\learn\線程調用
BASE_DIR = os.path.dirname(os.path.abspath('__file__'))

# =============== cmd 參數 ================
# 透過 cmd 呼叫
DIR = os.path.join(BASE_DIR, 'search.py')
cmd = "python " + f'"{DIR}"' + ' -v -p spam -p eggs foo.txt'
# ' -v -p spam --pat=eggs foo.txt bar.txt -o results --speed=fast'

# 印出 cmd DIR
print(cmd,BASE_DIR,sep='\n')

#  CONSOLE混雜
#os.system(cmd)
#subprocess.call(cmd)

#  NEW 一個 CONSOLE
subprocess.Popen(cmd, creationflags=subprocess.CREATE_NEW_CONSOLE)

# 暫停(需在要執行的 py 上)
#subprocess.call("pause",shell=True)
#os.system("pause")

python "D:\CODE\GitHub\py\資料結構\py3-cookbook\脚本编程与系统管理\search.py" -v -p spam -p eggs foo.txt
D:\CODE\GitHub\py\資料結構\py3-cookbook\脚本编程与系统管理


```py
ArgumentParser(
    prog=None, # program 的名字，None 就檔名
    usage=None, # None 就根據你設定的參數產生相對應的說明
    description=None, # 描述
    epilog=None # 補充說明 ex: epilog="see the doc: https://..."
    )
```

位置參數 (positional argument)

    parser.add_argument(dest='filenames',metavar='filename', nargs='*')
    
    # nargs='*' 任意位置(無前綴)的參數
     foo.txt

選擇性參數 (optional argument) 根據前綴來指定

    parser.add_argument('-o', dest='outfile', action='store',
                    help='output file')
                    
    -o results

# [getpass]运行时弹出密码输入提示

`Python` 的 `getpass` 模块正是你所需要的。

你可以让你很轻松的弹出密码输入提示， 并且不会在用户终端回显密码。

下面是具体代码

In [35]:
import getpass

user = getpass.getuser() # 'po390' # C:\Users\po390>
passwd = getpass.getpass()

def svc_login( passwd):
    if  passwd == "123":
        return True
    else:
        return False

if svc_login( passwd):    
    print('Yay!')
else:
    print('Boo!')
    
print(user)

········
Yay!
po390


如果你想显示的弹出用户名输入提示，使用内置的 `input` 函数：

```py
user = input('Enter your username: ')
```

# [os.get_terminal_size()]获取终端的大小

使用 `os.get_terminal_size()` 函数来做到这一点。

In [37]:
import os
sz = os.get_terminal_size()

sz
sz.columns
sz.lines

os.terminal_size(columns=120, lines=30)

120

30

# 执行外部命令并获取它的输出